In [1]:
import sqlalchemy
import db_module

create_new_test_db = True
import os
if os.path.exists("tests.sqlite"): #Если файл уже есть, заморачиваться с перезаполнением не будем
    create_new_test_db = False

## Создаем тестовую БД методом класса dbModule

In [2]:
myDB = db_module.dbModule(dbstring = 'sqlite:///tests.sqlite')
if create_new_test_db:
    myDB.create_sqlite_file()

## Напечатаем названия таблиц в БД для проверки

In [3]:
db_module.Base.metadata.tables.keys()

dict_keys(['image', 'dataset', 'category', 'license', 'annotation', 'trainResult', 'categoryToModel', 'model'])

## Заполним базу аннотациями из COCO -- это необходимо делать один раз

In [4]:
dataDir=r"D:\Work\AutoML\annotations_trainval2017"
dataType='train2017'
annFile=r"{}\annotations\instances_{}.json".format(dataDir,dataType)


import time

begin = time.time()
if create_new_test_db:
    myDB.fill_coco(annFile, firstTime = True) #Надо указать True чтобы подгрузить категории 
end = time.time()

print('DB filled in', end-begin, 'seconds')

DB filled in 0.0 seconds


In [5]:
myDB.add_model_record(task_type = "detection", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkPP', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'accuracy':0.80}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkPP', metrics = {'accuracy':0.93}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'loss':0.93}, history_address = '')

In [6]:
model = myDB.sess.query(db_module.dbModule.Model).join(db_module.dbModule.TrainResult).filter(db_module.dbModule.TrainResult.metric_name.in_(['loss','sd']))

In [7]:
model.all()

In [8]:
myDB.get_models_by_filter({'min_metrics':{'accuracy':0.92}, 'categories':['cat','mouse']})

,ID,model_address,task_type,aux,ID,metric_name,metric_value,model_id,history_address,aux
0,2,C:\Users\fires\repos\ann-automl\TrainedNN\New_...,classification,,4,accuracy,0.93,2,C:\Users\fires\repos\ann-automl,
1,2,C:\Users\fires\repos\ann-automl\TrainedNN\New_...,classification,,9,accuracy,0.93,2,C:\Users\fires\repos\ann-automl,
2,2,C:\Users\fires\repos\ann-automl\TrainedNN\New_...,classification,,14,accuracy,0.93,2,C:\Users\fires\repos\ann-automl,


In [11]:
myDB.load_specific_categories_annotations([15,16])

Train shape: (12386, 7)  test shape: (4129, 7) validation shape (4129, 7)


(              file_name                                           coco_url  \
 0      000000202273.jpg  http://images.cocodataset.org/train2017/000000...   
 1      000000475407.jpg  http://images.cocodataset.org/train2017/000000...   
 2      000000019608.jpg  http://images.cocodataset.org/train2017/000000...   
 3      000000469793.jpg  http://images.cocodataset.org/train2017/000000...   
 4      000000358765.jpg  http://images.cocodataset.org/train2017/000000...   
 ...                 ...                                                ...   
 20639  000000561931.jpg  http://images.cocodataset.org/train2017/000000...   
 20640  000000214131.jpg  http://images.cocodataset.org/train2017/000000...   
 20641  000000181515.jpg  http://images.cocodataset.org/train2017/000000...   
 20642  000000116123.jpg  http://images.cocodataset.org/train2017/000000...   
 20643  000000509691.jpg  http://images.cocodataset.org/train2017/000000...   
 
        category_id                             bb